## 수치 미분을 이용한 심층 신경망 학습

In [16]:
import time
import numpy as np

## 유틸리티 함수

In [17]:
epsilon = 0.0001

def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mean_squared_error(h, y):
    return 1 / 2 * np.mean(np.square(h - y))

## 뉴런 구현

In [18]:
class Neuron:
    def __init__(self, W, b, a):
        # Model Parameter
        self.W = W
        self.b = b
        self.a = a
        
        # Gradients
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)

    def __call__(self, x):
        return self.a(_m(_t(self.W), x) + self.b) # activation((W^T)x + b)

## 심층신경망 구현

In [25]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))

        self.sequence = list()
        # First hidden layer
        W, b = init_var(num_input, num_neuron)
        self.sequence.append(Neuron(W, b, activation))
        
        # Hidden layers
        for _ in range(hidden_depth - 1):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Neuron(W, b, activation))

        # Output layer
        W, b = init_var(num_neuron, num_output)
        self.sequence.append(Neuron(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, x, y, loss_func):
        def get_new_sequence(layer_index, new_neuron):
            new_sequence = list()
            for i, layer in enumerate(self.sequence):
                if i == layer_index:
                    new_sequence.append(new_neuron)
                else:
                    new_sequence.append(layer)
            return new_sequence
        
        def eval_sequence(x, sequence):
            for layer in sequence:
                x = layer(x)
            return x
        
        loss = loss_func(self(x), y)
        
        for layer_id, layer in enumerate(self.sequence): # iterate layer
            for w_i, w in enumerate(layer.W): # iterate W (row)
                for w_j, ww in enumerate(w): # iterate W (col)
                    W = np.copy(layer.W)
                    W[w_i][w_j] = ww + epsilon 
                    
                    new_neuron = Neuron(W, layer.b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x, new_seq)
                    
                    grad = (loss_func(h, y) - loss) / epsilon # (f(x +eps) -f(x)) / epsilon                    
                    layer.dW[w_i][w_j] = grad
                
                for b_i, bb in enumerate(layer.b): #iterate b
                    b = np.copy(layer.b)
                    b[b_i] = bb + epsilon 
                    
                    new_neuron = Neuron(layer.W, b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_neuron)
                    h = eval_sequence(x, new_seq)
                    
                    grad = (loss_func(h, y) - loss) / epsilon # (f(x +eps) -f(x)) / epsilon                    
                    layer.db[b_i] = grad
        return loss

## 경사하강 학습법

In [26]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = network.calc_gradient(x, y, loss_obj)
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

## 동작 테스트

In [27]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

dnn = DNN(hidden_depth=5, num_neuron=32, num_input=10, num_output=2, activation=sigmoid)

t = time.time()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, mean_squared_error, 0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 0.12582883508358192
Epoch 1: Test loss 0.12512219245726067
Epoch 2: Test loss 0.12441970702033667
Epoch 3: Test loss 0.12372138250602982
Epoch 4: Test loss 0.12302722184822201
Epoch 5: Test loss 0.12233722719396994
Epoch 6: Test loss 0.12165139991625486
Epoch 7: Test loss 0.12096974062691093
Epoch 8: Test loss 0.12029224918975881
Epoch 9: Test loss 0.11961892473385972
Epoch 10: Test loss 0.11894976566694397
Epoch 11: Test loss 0.11828476968895318
Epoch 12: Test loss 0.1176239338057081
Epoch 13: Test loss 0.1169672543426679
Epoch 14: Test loss 0.11631472695876258
Epoch 15: Test loss 0.11566634666032036
Epoch 16: Test loss 0.1150221078150281
Epoch 17: Test loss 0.11438200416594003
Epoch 18: Test loss 0.11374602884552344
Epoch 19: Test loss 0.11311417438971097
Epoch 20: Test loss 0.11248643275194503
Epoch 21: Test loss 0.11186279531725293
Epoch 22: Test loss 0.11124325291626061
Epoch 23: Test loss 0.11062779583919384
Epoch 24: Test loss 0.11001641384983613
Epoch 25: Tes